In [8]:
# ===============================
# Telecom X Challenge – ETL y EDA Completos
# ===============================

import pandas as pd, ast, os, json
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import json_normalize
from scipy.stats import chi2_contingency

# 1. Cargar datos desde GitHub (API)
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"
df = pd.read_json(url)

# 2. Expandir columnas con diccionarios
# for col in ["customer", "phone", "internet", "account"]:
#     df[col] = df[col].apply(ast.literal_eval) # This line is causing the error
dfs = [json_normalize(df[col]) for col in ["customer", "phone", "internet", "account"]]
df_expanded = pd.concat([df[["customerID", "Churn"]]] + dfs, axis=1)

# 3. Convertir nombres de columnas a snake_case
def clean_name(c): return c.strip().lower().replace(' ', '_').replace('-', '_')
df_expanded = df_expanded.rename(columns=lambda x: clean_name(x))

# 4. Revisión de estructura y nulos
print(">>> Estructura del DataFrame:")
print(df_expanded.info(), "\n")
print("Primeras filas:")
display(df_expanded.head())

# 5. Limpiar y convertir formatos
df_expanded["churn"] = df_expanded["churn"].str.lower().map({"yes":1, "no":0}).fillna(-1).astype(int)
# Opcional: convertir numeric columns if needed
for c in ["charges.monthly", "charges.total", "tenure"]:
    if c in df_expanded.columns:
        df_expanded[c] = pd.to_numeric(df_expanded[c], errors='coerce')
# TotalCharges con NaN donde tenure == 0 → reemplazar por 0
if "charges.total" in df_expanded.columns and "tenure" in df_expanded.columns:
    df_expanded.loc[df_expanded["tenure"]==0, "charges.total"] = df_expanded.loc[df_expanded["tenure"]==0, "charges.total"].fillna(0)


# 6. Ingeniería de features
df_expanded["charges_daily_est"] = df_expanded["charges.monthly"] / 30
df_expanded["charges_total_daily_est"] = df_expanded.apply(
    lambda r: r["charges.total"] / (r["tenure"]*30) if r["tenure"]>0 else r["charges.total"], axis=1
)
# Contar número de servicios (assuming binary indicators)
service_cols = [c for c in df_expanded.columns if c in (
    ["phoneservice", "multiplelines", "onlinesecurity", "onlinebackup", "deviceprotection",
     "techsupport", "streamingtv", "streamingmovies"])]
df_expanded["n_services"] = df_expanded[service_cols].apply(lambda row: sum(row== "yes") if row.dtype=='object' else sum(row), axis=1)


# 7. Guardar el dataset limpio
os.makedirs("data_processed", exist_ok=True)
df_expanded.to_csv("data_processed/telecomx_model_ready.csv", index=False)

# 8. EDA – descriptivo
churn_rate = df_expanded["churn"].mean()
print(f"\nChurn rate: {churn_rate:.2%}")

plt.figure(figsize=(6,4))
sns.countplot(x="churn", data=df_expanded)
plt.title("Distribución de Churn (0=no, 1=sí)")
plt.savefig("churn_count.png")
plt.close()

# Boxplots tenure y monthly charges vs churn
for col in ["tenure", "charges.monthly"]:
    if col in df_expanded.columns:
        plt.figure(figsize=(8,5))
        sns.boxplot(x="churn", y=col, data=df_expanded)
        plt.title(f"{col.title()} por Churn")
        plt.savefig(f"{col}_by_churn.png")
        plt.close()

# Correlación numérica
num_cols = df_expanded.select_dtypes(include="number").columns.drop("churn")
corr = df_expanded[num_cols.union(["churn"])].corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="RdBu", center=0)
plt.title("Matriz de Correlación (numéricas con churn)")
plt.savefig("corr_matrix.png")
plt.close()

# 9. Pruebas estadísticas en categóricas selectors (ej. contract, paymentmethod)
cat_cols = ["contract", "paymentmethod", "internetservice"]
cat_results = {}
for c in cat_cols:
    if c in df_expanded.columns:
        cont = pd.crosstab(df_expanded[c], df_expanded["churn"])
        if cont.shape[0]>1:
            chi2, p, _, _ = chi2_contingency(cont)
            cat_results[c] = p
print("\nP-valores chi2 categóricas:")
print(cat_results)

# 10. Generar plantilla de informe en Markdown
with open("report.md", "w") as f:
    f.write("# Informe - Challenge Telecom X\n\n")
    f.write("## 1. Objetivo\nAnalizar causas del churn y preparar datos para modelado.\n\n")
    f.write("## 2. Hallazgos principales\n")
    f.write(f"- Tasa de churn: **{churn_rate:.2%}**\n")
    if not corr.empty:
        top_corr = corr["churn"].abs().sort_values(ascending=False).head(5).to_dict()
        f.write("- Variables numéricas más correlacionadas con churn:\n")
        for k,v in top_corr.items():
            f.write(f"  - {k}: {v:.2f}\n")
    if cat_results:
        f.write("- Variables categóricas con asociación significativa (p<0.05):\n")
        for k,v in cat_results.items():
            f.write(f"  - {k}: p-value = {v:.4f}\n")
    f.write("\n## 3. Metodología\n- Extracción desde API → JSON.\n- Limpieza y transformación → columnas expandidas, tipos numéricos, features diarias.\n- Análisis exploratorio: tasas, correlaciones, pruebas chi-cuadrado.\n\n")
    f.write("## 4. Visualizaciones generadas\n- churn_count.png\n- tenure_by_churn.png, monthlycharges_by_churn.png (si existen)\n- corr_matrix.png\n\n")
    f.write("## 5. Recomendaciones\n- Enfocar retención en clientes con baja tenure y altos cargos diarios.\n- Revisar métodos de pago o contratos con alta evasión.\n\n")
    f.write("## 6. Siguientes pasos\n- Entrenar modelos predictivos (logistic regression, random forest).\n- Evaluar importancia de variables y aplicar estrategias segmentadas.\n")

print("✅ Cuaderno completo generado. Archivos: telecomx_model_ready.csv, report.md y gráficos PNG.")

>>> Estructura del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerid        7267 non-null   object 
 1   churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   seniorcitizen     7267 non-null   int64  
 4   partner           7267 non-null   object 
 5   dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   phoneservice      7267 non-null   object 
 8   multiplelines     7267 non-null   object 
 9   internetservice   7267 non-null   object 
 10  onlinesecurity    7267 non-null   object 
 11  onlinebackup      7267 non-null   object 
 12  deviceprotection  7267 non-null   object 
 13  techsupport       7267 non-null   object 
 14  streamingtv       7267 non-null   object 
 15  streamingmovies   7267 non-null   object 
 16  contract    

,customerid,churn,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,...,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,charges.monthly,charges.total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4



Churn rate: 22.64%

P-valores chi2 categóricas:
{'contract': np.float64(1.3605128871670327e-255), 'paymentmethod': np.float64(8.364832403522647e-137), 'internetservice': np.float64(2.2226611231770693e-157)}
✅ Cuaderno completo generado. Archivos: telecomx_model_ready.csv, report.md y gráficos PNG.
